In [3]:
import pandas as pd
import numpy as np
import requests


# Web Scrapping

Muitas vezes os dados que queremos não estão disponibilizados através de APIs, apenas sites. Neste momento precisamos recorrer às ferramentas de **Web Scrapping**!

*Web scrapping* é a extração de informação estruturada a partir de paginas na internet: por exemplo, podemos extrair todos os artigos de um jornal que mencionem um certo produto, ou então as informações de uma série de tabelas da Wikipedia.

Hoje vamos aprender como utilizar as bibliotecas BeautifulSoup e Selenium para extrair informações a partir de links específicos, realizar buscas e navegar páginas.

# Conhecendo o BeautifulSoup

Vamos começar extraindo informações básicas a partir de uma notícia do portal UOL. O primeiro passo é utilizar a biblioteca `requests` para *baixar* o html da página:

In [1]:
url = "https://www.uol.com.br/esporte/futebol/ultimas-noticias/2022/01/22/em-1995-decisao-na-base-entre-palmeiras-x-sp-terminou-em-morte-no-pacaembu.htm"


In [4]:
response = requests.get(url)
html_str = response.text
type(html_str)


str

In [8]:
print(html_str[0:1000])


<!DOCTYPE html> <html lang="pt-br"> <head><meta charset="utf-8"><meta http-equiv="Content-Type" content="text/html; charset=utf-8">   <title>Em 1995, decisão na base entre Palmeiras x SP terminou em morte no Pacaembu - 22/01/2022 - UOL Esporte</title><link rel="preconnect" href="https://stc.uol.com" crossorigin="anonymous"><link rel="preconnect" href="https://c.jsuol.com.br" crossorigin="anonymous"><link rel="preconnect" href="https://conteudo.jsuol.com.br" crossorigin="anonymous"><link rel="preconnect" href="https://conteudo.imguol.com.br" crossorigin="anonymous"><link rel="preconnect" href="https://me.jsuol.com.br" crossorigin="anonymous"><link rel="preconnect" href="https://www.google-analytics.com" crossorigin="anonymous"><link rel="dns-prefetch" href="https://stc.uol.com"><link rel="dns-prefetch" href="https://c.jsuol.com.br"><link rel="dns-prefetch" href="https://conteudo.jsuol.com.br"><link rel="dns-prefetch" href="https://conteudo.imguol.com.br"><link rel="dns-prefetch" href="h

In [9]:
html_bytes = response.content
type(html_bytes)


bytes

In [10]:
print(html_bytes[0:1000])


b'<!DOCTYPE html> <html lang="pt-br"> <head><meta charset="utf-8"><meta http-equiv="Content-Type" content="text/html; charset=utf-8">   <title>Em 1995, decis\xc3\xa3o na base entre Palmeiras x SP terminou em morte no Pacaembu - 22/01/2022 - UOL Esporte</title><link rel="preconnect" href="https://stc.uol.com" crossorigin="anonymous"><link rel="preconnect" href="https://c.jsuol.com.br" crossorigin="anonymous"><link rel="preconnect" href="https://conteudo.jsuol.com.br" crossorigin="anonymous"><link rel="preconnect" href="https://conteudo.imguol.com.br" crossorigin="anonymous"><link rel="preconnect" href="https://me.jsuol.com.br" crossorigin="anonymous"><link rel="preconnect" href="https://www.google-analytics.com" crossorigin="anonymous"><link rel="dns-prefetch" href="https://stc.uol.com"><link rel="dns-prefetch" href="https://c.jsuol.com.br"><link rel="dns-prefetch" href="https://conteudo.jsuol.com.br"><link rel="dns-prefetch" href="https://conteudo.imguol.com.br"><link rel="dns-prefetch

## Transfome em sopa

A página extraída acima é apenas um string: o código HTML. Para utiliza-la dentro do Python, precisamos de um **parser**: um conjunto de funções que nos permite **interpretar** este código HTML e extrair informações relevantes. A biblioteca *BeautifulSoup* implementa um **parser** de HTML dentro do Python, dando acesso à arvore de tags (`<head>`, `<link ...> `, etc).

Para utilizar este **parser** precisamos entender um pouco da estrutura de um arquivo HTML. Mas antes vamos utilizar o BeautifulSoup para deixar o *print* de nosso HTML mais organizado:

In [ ]:
!pip3 install bs4

In [16]:
from bs4 import BeautifulSoup


In [17]:
soup = BeautifulSoup(html_bytes)


In [18]:
type(soup)


bs4.BeautifulSoup

In [19]:
print(soup.prettify())


<!DOCTYPE html>
<html lang="pt-br">
 <head>
  <meta charset="utf-8"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Em 1995, decisão na base entre Palmeiras x SP terminou em morte no Pacaembu - 22/01/2022 - UOL Esporte
  </title>
  <link crossorigin="anonymous" href="https://stc.uol.com" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://c.jsuol.com.br" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://conteudo.jsuol.com.br" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://conteudo.imguol.com.br" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://me.jsuol.com.br" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://www.google-analytics.com" rel="preconnect"/>
  <link href="https://stc.uol.com" rel="dns-prefetch"/>
  <link href="https://c.jsuol.com.br" rel="dns-prefetch"/>
  <link href="https://conteudo.jsuol.com.br" rel="dns-prefetch"/>
  <link href="https://conteudo.imguol.com.

## Encontrando Tags

Um arquivo HTML é estruturado em **tags**: marcações com o formato `<nome_do_tag>` e `</nome_do_tag>`. A primeira denota o **inicio do conteúdo** do tag, a segunda o **fim do conteúdo**. Vamos entender como um **tag simples** funciona analisando o conteúdo do `<title>`.

Para encontrar todas as ocorrências de um **tag** pelo seu **nome** utilizamos o método `.find_all()`:

In [21]:
soup.find_all("title")


[<title>Em 1995, decisão na base entre Palmeiras x SP terminou em morte no Pacaembu - 22/01/2022 - UOL Esporte</title>]

O método nos retornou todas as ocorrências de `<title>` no HTML (uma só no caso) em uma lista. Vamos ver o que essa lista contém:

In [22]:
title = soup.find_all("title")[0]
print(type(title))


<class 'bs4.element.Tag'>


O objeto `Tag` da BeautifulSoup contém todas as informações de um tag: atributos, links, conteúdo... Por enquanto vamos olhar o conteúdo desse tag (o que está entre `<title>` e `< \title>`) utilizando o atributo `.text`:

In [23]:
title.text


'Em 1995, decisão na base entre Palmeiras x SP terminou em morte no Pacaembu - 22/01/2022 - UOL Esporte'

In [24]:
type(title.text)


str

### **Tags** com mais que uma ocorrência 

A maior parte dos **tags** ocorrem múltiplas vezes em um documento. Vamos buscar um **tag** com essa característica.

In [36]:
headers = soup.find_all("h3")
print(headers)


[<h3 class="thumb-title"> Romário e Marcelle falam sobre fim de namoro e negam traição  </h3>, <h3 class="thumb-title"> Juca Kfouri: Brasil inteiro vai torcer pelo Flamengo contra o Palmeiras  </h3>, <h3 class="thumb-title"> Vídeo mostra lutador Leandro Lo agredindo motoboy ao sair de bar em SP  </h3>, <h3>Ocorreu um erro ao carregar os comentários.</h3>, <h3>Essa discussão está encerrada</h3>, <h3 class="h-components collection-title custom-title">Futebol</h3>, <h3 class="thumb-title title-xsmall title-lg-small">Está difícil achar o álbum da Copa do Mundo? Veja como conseguir o seu</h3>, <h3 class="thumb-title title-xsmall title-lg-small">Em reta final de contrato, Athletico encaminha contratação de atacante chileno</h3>, <h3 class="thumb-title title-xsmall title-lg-small">'O pastor está voltando?': Desfalque do São Paulo há dois meses, André Anderson faz postagem misteriosa</h3>, <h3 class="thumb-title title-xsmall title-lg-small">Celta de Vigo x Real Madrid: onde assistir, horário e

In [37]:
print(headers[0])


<h3 class="thumb-title"> Romário e Marcelle falam sobre fim de namoro e negam traição  </h3>


In [38]:
print(headers[0].text)


 Romário e Marcelle falam sobre fim de namoro e negam traição  


In [39]:
# EXERCICIO
# Utilize uma list comprehension para criar uma lista com o texto de todos os headers h3


### Buscando múltiplos **tags**

Além de buscar **tags** um a um, podemos utilizar o método `.find_all()` para encontrar todas as ocorrências de uma lista de tags:

In [40]:
lista_tags = ["h1", "h2", "h3"]
todos_headers = soup.find_all(lista_tags)
print(todos_headers)


[<h2 class="title-name"> <a data-audience-click='{"reference":"titulo-colecao","component":"title","mediaName":"Title"}' href="https://www.uol.com.br/esporte/futebol/ultimas/">Futebol</a> </h2>, <h1 class=""> <span> <i class="col-sm-22 col-md-22 col-lg-22 custom-title" ia-title="">Em 1995, decisão na base entre Palmeiras x SP terminou em morte no Pacaembu  </i></span> </h1>, <h3 class="thumb-title"> Romário e Marcelle falam sobre fim de namoro e negam traição  </h3>, <h3 class="thumb-title"> Juca Kfouri: Brasil inteiro vai torcer pelo Flamengo contra o Palmeiras  </h3>, <h3 class="thumb-title"> Vídeo mostra lutador Leandro Lo agredindo motoboy ao sair de bar em SP  </h3>, <h3>Ocorreu um erro ao carregar os comentários.</h3>, <h3>Essa discussão está encerrada</h3>, <h3 class="h-components collection-title custom-title">Futebol</h3>, <h3 class="thumb-title title-xsmall title-lg-small">Está difícil achar o álbum da Copa do Mundo? Veja como conseguir o seu</h3>, <h3 class="thumb-title titl

Podemos utilizar o atributo `.name` para determinar qual o tipo de cada tag em nossa lista:

In [50]:
print(todos_headers[0].name)
print(todos_headers[0].text)


h2
 Futebol 


In [51]:
# EXERCICIO
# Utilize uma list comprehension para criar uma lista
# de uplas (tipo do tag, conteúdo)


### Buscando links

Um **tag** específico muito útil na construção de **web crawlers** é o `<a>`. Este **tag** contém os hiper-links de uma página HTML. Vamos utilizar a BeautifulSoup para extrair todas os links de nossa notícia.

In [54]:
tag_a = soup.find_all("a")


In [55]:
tag_a[0:5]


[<a data-audience-click="" href="https://www.ingresso.com/?utm_source=uol.com.br&amp;utm_medium=barrauol&amp;utm_campaign=linkfixo_barrauol&amp;utm_content=barrauol-link-ingressocom&amp;utm_term=barrauol-ingressocom">Ingresso.com</a>,
 <a data-audience-click='{"component":"barra-uol","reference":"item-batepapo","position":"coluna-unica","area":"barra-uol","mediaName":"Home"}' href="https://batepapo.uol.com.br/?utm_source=midia-interna_uol.com.br&amp;utm_medium=barrauol-internas&amp;utm_campaign=linkfixo_barrauol&amp;utm_term=barrauol-uolplay&amp;utm_content=barrauol">BATE-PAPO</a>,
 <a data-audience-click="" href="https://meunegocio.uol.com.br/?utm_source=uol.com.br&amp;utm_medium=barrauol&amp;utm_campaign=linkfixo-barrauol-umn&amp;utm_term=barrauol-umn&amp;utm_content=barrauol-umn">MEU NEGÓCIO</a>,
 <a data-audience-click="" href="https://www.passeidireto.com/?utm_source=uol.com.br&amp;utm_medium=barra-uol-interna">Passei Direto</a>,
 <a data-audience-click='{"component":"barra-uol","

In [56]:
first_link = tag_a[0]


In [57]:
first_link


<a data-audience-click="" href="https://www.ingresso.com/?utm_source=uol.com.br&amp;utm_medium=barrauol&amp;utm_campaign=linkfixo_barrauol&amp;utm_content=barrauol-link-ingressocom&amp;utm_term=barrauol-ingressocom">Ingresso.com</a>

O atributo `.text` não extrai o URL do link, apenas o texto que é exibido para o usuário:

In [59]:
first_link.text


'Ingresso.com'

Se olharmos o *string* do **tag** poderemos entender melhor porque isso acontece. O URL em si está **dentro da declaração do tag**:

    <a data-audience-click="" href="https://www.ingresso.com/?utm_source=uol.com.br&amp;utm_medium=barrauol&amp;utm_campaign=linkfixo_barrauol&amp;utm_content=barrauol-link-ingressocom&amp;utm_term=barrauol-ingressocom">

Dentro da declaração do **tag** podemos ver algo parecido com a declaração de variáveis:

    data-audience-click=""

e

    href="https://www.ingresso.com/?utm_source=uol.com.br&amp;utm_medium=barrauol&amp;utm_campaign=linkfixo_barrauol&amp;utm_content=barrauol-link-ingressocom&amp;utm_term=barrauol-ingressocom"

Cada uma dessas *variáveis* é um **atributo do tag** e para extrai-las utilizaremos o atributo `.attrs`:

In [60]:
first_link.attrs


{'href': 'https://www.ingresso.com/?utm_source=uol.com.br&utm_medium=barrauol&utm_campaign=linkfixo_barrauol&utm_content=barrauol-link-ingressocom&utm_term=barrauol-ingressocom',
 'data-audience-click': ''}

Podemos ver que os atributos de um **tag** são retornados como um **dict**! Se quisermos acessar uma **variável** específica podemos faze-lo através do nome desta variável:

In [61]:
first_link.attrs["href"]


'https://www.ingresso.com/?utm_source=uol.com.br&utm_medium=barrauol&utm_campaign=linkfixo_barrauol&utm_content=barrauol-link-ingressocom&utm_term=barrauol-ingressocom'

In [63]:
# EXERCICIO
# Utilize um loop para extrair todos os URLs de
# tags <a>


## **Tags** hierárquicos

Até agora todos os **tags** que vimos eram **simples**, ou seja, não continham em seu conteúdo outros **tags**. Muitas vezes queremos extrair informações *navegando* a página **bloco a bloco**.

Vamos aprender a utilizar o **inspetor de código** de um web browser para navegar blocos de tags para extrair informações complexas. Neste exemplo vamos reconstruir um tabela da Wikipedia com um DataFrame.

In [64]:
url = "https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy"


In [65]:
response = requests.get(url)
html = response.content


In [66]:
soup = BeautifulSoup(html)


In [67]:
print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of European countries by life expectancy - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"d5a42ee4-9457-4303-94fa-c05c4fd1aaf4","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_European_countries_by_life_expectancy","wgTitle":"List of European countries by life expectancy","wgCurRevisionId":1104436050,"wgRevisionId":1104436050,"wgArticleId":22175559,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from April 2022","Articl

Vamos acessar o link [List of European countries by life_expectancy](https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy) para entender como podemos utilizar o inspetor de código fonte para descobrir onde em nosso HTML está nossa tabela!

### Extraindo a tabela completa

Com o inspetor podemos ver que as tabelas desta página estão todas dentro de tags `table`. 

In [78]:
table_wiki_raw = soup.find_all("table")


In [79]:
len(table_wiki_raw)


13

Infelizmente, esta **tag** é muitas vezes utilizada como elemento de formatação de páginas (especialmente em página antigas).

Podemos utilizar o **argumento** `attrs =` do método `.find_all()` para **filtrar** os **tags** extraídos a partir dos **valores de seus atributos**.

O atributo `class` é um ótimo candidato em varias ocasiões para esse tipo de filtro. No caso da nossa tabela atual podemos ver que a classe é:

    wikitable sortable static-row-numbers plainrowheaders srn-white-background jquery-tablesorter

Vamos ver como utilizar o `attrs =` para realizar esse filtro.

In [84]:
table_wiki_raw = soup.find_all("table", attrs={"class": "wikitable"})


In [85]:
len(table_wiki_raw)


4

In [86]:
print(table_wiki_raw[0])


<table border="1" class="wikitable sortable static-row-numbers plainrowheaders srn-white-background" style="text-align:right;">
<tbody><tr class="static-row-header" style="text-align:center;vertical-align:bottom;">
<th>Countries
</th>
<th style="width:4em;">all
</th>
<th style="width:4em;">male
</th>
<th style="width:4em;">female
</th>
<th style="width:4em;"><abbr title="Difference in life expectancy between females and males">gender<br/>gap</abbr>
</th>
<th style="width:4em; border-left-width:2px;"><abbr title="Improvement of the indicator for all population compared to the previous year">Δ 2019<br/>all</abbr>
</th>
<th style="width:4em;"><abbr title="Improvement of the indicator for male compared to the previous year">Δ 2019<br/>male</abbr>
</th>
<th style="width:4em;"><abbr title="Improvement of the indicator for female compared to the previous year">Δ 2019<br/>female</abbr>
</th>
<th style="width:5em;"><abbr title="Increase of gender gap compared to the previous year">Δ 2019<br/>ge

### Navegando um **tag hierárquico**

Além de poder percorrer o HTML completo utilizando o método `.find_all()`, podemos fazer buscas dentro de cada **tag**! Isso nos permite **localizar** a busca em um bloco específico delimitado por **tags**.

No nosso caso atual, podemos ver que cada tabela extraída é composta por **três tags**: `<thead>`, `<tbody>` e `<tfoot>`. Como queremos extrair o **corpo** da tabela, vamos investigar o tag `tbody`

In [87]:
table_wiki = table_wiki_raw[3].find("tbody")


In [89]:
print(table_wiki.prettify())


<tbody>
 <tr bgcolor="#efefef">
  <th>
   Rank
  </th>
  <th>
   Country
  </th>
  <th>
   <a href="/wiki/List_of_countries_by_life_expectancy" title="List of countries by life expectancy">
    Life expectancy
   </a>
   <sup class="reference" id="cite_ref-:0_6-1">
    <a href="#cite_note-:0-6">
     [6]
    </a>
   </sup>
  </th>
  <th>
   Influenza vaccination rate, people aged 65 and over, 2016 (%)
   <sup class="reference" id="cite_ref-7">
    <a href="#cite_note-7">
     [7]
    </a>
   </sup>
  </th>
 </tr>
 <tr>
  <td>
   1
  </td>
  <td>
   <span class="flagicon">
    <img alt="" class="thumbborder" data-file-height="800" data-file-width="1000" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Flag_of_Monaco.svg/19px-Flag_of_Monaco.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Flag_of_Monaco.svg/29px-Flag_of_Monaco.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Flag_of_Monaco.svg/38px-Flag_of_Monaco.s

Podemos ver tanto no inspetor quanto no print acima que o corpo da tabela é composto por múltiplos tags `<tr>`: se olharmos com cuidado veremos que cada `<tr>` é uma linha de nossa tabela! Vamos extrair uma linha em particular para ver do que ela é composta.

In [90]:
table_rows = table_wiki.find_all("tr")


In [91]:
print(table_rows[1].prettify())


<tr>
 <td>
  1
 </td>
 <td>
  <span class="flagicon">
   <img alt="" class="thumbborder" data-file-height="800" data-file-width="1000" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Flag_of_Monaco.svg/19px-Flag_of_Monaco.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Flag_of_Monaco.svg/29px-Flag_of_Monaco.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Flag_of_Monaco.svg/38px-Flag_of_Monaco.svg.png 2x" width="19"/>
  </span>
  <a href="/wiki/Monaco" title="Monaco">
   Monaco
  </a>
  <sup class="reference" id="cite_ref-8">
   <a href="#cite_note-8">
    [8]
   </a>
  </sup>
 </td>
 <td>
  89.4
 </td>
 <td>
 </td>
</tr>



Cada linha de nossa tabela é composta por uma série de tags `<td>` - e cada um destes contém as informações de uma célula. Finalmente chegamos no elemento que contém os dados da tabela!

In [92]:
first_row = table_rows[1].find_all("td")


In [95]:
first_row[2].text


'89.4\n'

In [96]:
[elemento.text.strip() for elemento in first_row]


['1', 'Monaco[8]', '89.4', '']

Vamos juntar todas as etapas acima em um loop (ou list comprehension) para construir nosso DataFrame!

In [ ]:
# EXERCICIO
# Construir um DataFrame com os dados da tabela de Expectativa de Vida na Europa da Wikipedia
# BONUS - faça isso com um list comprehension!
# BONUS - contrua uma função que receba um link da Wikipedia com uma tabela e retorne um DataFrame (teste em outra tabela)


In [ ]:
pd.DataFrame(
    [
        [elemento.text.strip() for elemento in first_row.find_all("td")]
        for first_row in table_rows
    ],
    columns=["rank", "pais", "ev", "vacc"],
)


In [ ]:
table = []
for row in table_rows:
    row_tds = row.find_all("td")
    row_list = []

    for td in row_tds:
        row_list.append(td.text.strip())

    table.append(row_list)


# Criando um WebCrawler

Um **webcrawler** é uma aplicação que extrai informações estruturadas a partir de multiplos sites de forma autonôma. Vamos construir um **webcrawler** para extrair um artigo (artigo origem) do jornal Guardian e qualquer outro artigo referenciado no texto do artigo origem (artigos filhos).

## Extraindo o Artigo Origem

O primeiro passo do nosso *crawler* é encontrar o corpo principal do artigo - caso contrário *puxaremos* links de propagandas, artigos relacionados, cabeçalho, etc...

In [106]:
url = "https://www.theguardian.com/world/2022/aug/18/fires-and-explosions-reported-at-military-targets-in-russia-and-crimea"
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html)


Vamos começar buscando algum **tag** que represente o corpo do artigo:

In [107]:
artigo = soup.find("article")
print(artigo.text)


RussiaFires and explosions reported at military targets in Russia and CrimeaMunitions depot in Belgorod province and airbase near Sevastopol hit in latest apparent sabotage missions

See all our Ukraine war coverage
00:49Russia: huge fire at ammunition depot near border with Ukraine – videoEmma Graham-Harrison in KyivFri 19 Aug 2022 18.08 BSTFirst published on Thu 18 Aug 2022 23.55 BSTFires and explosions have been reported at military targets inside Russia and Russian-occupied parts of Ukraine, in the latest of a string of apparent sabotage missions deep inside Russian-held territory as western officials suggested the conflict had reached deadlock.Two Russian villages were evacuated after a blaze at a munitions depot near the Ukrainian border in Belgorod province. “An ammunition depot caught fire near the village of Timonovo”, less than 30 miles (50km) from the border, the regional governor, Vyacheslav Gladkov, said in a statement, adding that no casualties had been reported.At least 

Agora que encontramos um **tag** com o corpo do artigo precisamos encontrar links dentro deste corpo:

In [108]:
links_artigo = artigo.find_all("a", attrs={"data-link-name": "in body link"})
len(links_artigo)


4

Podemos utilizar list comprehensions para visualizar para onde esses links nos levarão:

In [109]:
[link["href"] for link in links_artigo]


['https://www.theguardian.com/world/2022/aug/16/ukraine-hints-it-was-behind-latest-attack-on-russian-supply-lines-in-crimea',
 'https://www.theguardian.com/world/2022/aug/10/ukraine-air-force-claims-russian-jets-destroyed-crimea-raid',
 'https://www.theguardian.com/world/2022/aug/16/ukraine-hints-it-was-behind-latest-attack-on-russian-supply-lines-in-crimea',
 'https://www.theguardian.com/world/volodymyr-zelenskiy']

Sempre que precisamos construir um loop para executar um bloco de código sobre os elementos de uma lista devemos **testar** esse bloco de código em um elemento particular da lista antes de contruir o loop completo:

In [110]:
url_filho = links_artigo[0]["href"]


Da mesma forma que extraímos o artigo original, podemos extrair o artigo filho a partir do link extraído:

In [111]:
response_filho = requests.get(url_filho)
html_filho = response_filho.content
soup_filho = BeautifulSoup(html_filho)


In [112]:
artigo_filho = soup_filho.find("article").text
print(artigo_filho)


UkraineUkraine hints it was behind latest attack on Russian supply lines in CrimeaWhile not formally confirming responsibility for mystery strike, Kyiv officials react with glee on social media

Russia-Ukraine war: latest news
00:45Footage purports to show explosion at ammunition depot in Crimea – videoLuke Harding in KyivTue 16 Aug 2022 15.56 BSTFirst published on Tue 16 Aug 2022 13.22 BSTUkraine has hinted it was behind a series of mysterious and devastating strikes in occupied Crimea that destroyed a key railway junction used for supplying Russian troops and a military airbase.Smoke billowed into the sky near Dzhankoi, a significant railway hub in the north of the peninsula used by the Russian military to transport troops and equipment to occupied Melitopol, which Moscow seized early in its full-scale invasion.Several explosions on Tuesday appeared to have destroyed a Russian ammunition depot and an electricity substation about 125 miles (200km) from the frontline with Ukrainian for

Agora podemos consolidar nosso código para executar o bloco acima sobre todos os *links* extraídos do artigo origem (adicionando algumas camadas de segurança para que nosso *crawler* navegue tranquilamente):

In [ ]:
lista_artigos = []
lista_links = [link["href"] for link in links_artigo]
for link in lista_links:
    if link.find("guardian") >= 0:
        try:
            response_filho = requests.get(link)
            html_filho = response_filho.content
            soup_filho = BeautifulSoup(html_filho)
            lista_artigos.append(soup_filho.find("article").text)
        except AttributeError:
            continue


In [ ]:
lista_artigos[2]


In [ ]:
# EXERCICIO
# Transforme o código acima em uma função que recebe um link de artigo do Guardian como argumento
# e retorna uma lista com o texto do artigo original e o texto de quaisquer artigos do Guardian
# linkados no texto do artigo original
# BONUS - crie uma função que faça isso em profundidade maior que um: além do artigo original e dos artigos
# filhos (artigos linkados no artigo original) trazer os artigos netos (artigos linkados nos artigos filhos)
# BONUS BONUS - contrua uma função capaz de extrair uma profundidade arbitraria de artigos (filhos, netos, bisnetos...)
# a partir de um parametro 'depth' (0 = original, 1 = filhos, 2 = netos, 3 = bisnetos, etc)


# Conhecendo o Selenium

Embora o BeautifulSoup seja uma biblioteca ótima e simples para extrair dados de páginas (o complicado são os html's...), nem sempre conseguimos usa-la: muitas páginas utilizam, hoje em dia, tecnologias incompatíveis com a arquitetura do BeautifulSoup.

Páginas que são dinâmicas (especificamente, páginas que usem tecnologias client-side, como React.js) não podem ser mineradas diretamente.

Para tratar destas páginas precisamos utilizar outra biblioteca: Selenium. Enquanto na BeautifulSoup carregamos o HTML da página original para dentro do Python, com a Selenium **simularemos o ato de navegação**.

In [113]:
!pip3 install selenium
!pip3 install webdriver-manager

     |████████████████████████████████| 985 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 358 kB 13.4 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 10.8 MB/s eta 0:00:01


In [114]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


Com a Selenium, precisamos utilizar um WebDriver - um navegador específico através do qual extraíremos as informações desejadas. Para nossa aula de hoje vamos utilizar o Chrome:

In [117]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


[WDM] - Downloading: 100%|██████████| 8.04M/8.04M [00:00<00:00, 9.86MB/s]
/var/folders/c2/4vv1dtt97gl9bn1lvw8rf0k40000gn/T/ipykernel_38857/857920997.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


Vamos utilizar nosso `driver` para navegar até uma página teste:

In [118]:
driver.get("https://testpages.herokuapp.com/styled/index.html")


## Navegando utilizando XPATHs

O forma mais fácil de navegar uma página utilizando Selenium é através de XPATHs: identificadores únicos dos elementos de uma página. Vamos ver como podemos descobrir um XPATH de um elemento particular e como utilizar o nosso `driver` para interagir com o elemento.

In [119]:
link_htmlformtest = driver.find_element(
    By.XPATH, "/html/body/div/ul[1]/li[7]/ul/li[1]/a"
)


O método `.find_element()` nos permite encontrar elementos em uma página (parecido com o `.find_all()` do BS) - mas por si só não nos diz muito sobre o elemento:

In [123]:
link_htmlformtest


<selenium.webdriver.remote.webelement.WebElement (session="561981cdc99b53faa84c240fe6def5d7", element="3e962d98-6fec-47c7-a4f5-8621d20ed7aa")>

Podemos buscar os atributos do elemento utilizando o método `.get_attribute()` - vamos utilizar este método para extrair o URL do link associado ao elemento:

In [121]:
link_htmlformtest.get_attribute("href")


'https://testpages.herokuapp.com/styled/basic-html-form-test.html'

Até agora tudo está muito parecido com a BS - agora vamos interagir com a página, *clicando* virtualmente no link:

In [124]:
link_htmlformtest.click()


### Interagindo com Formulários

Um dos principais usos para o Selenium é conseguir preencher de forma programática formulários em páginas. Vamos continuar utilizando nosso driver para preencher o formulário na página para qual navegamos.

In [125]:
input_username = driver.find_element(
    By.XPATH, "/html/body/div/div[3]/form/table/tbody/tr[1]/td/input"
)


Podemos utilizar o método `.send_keys()` para *preencher* um formulário: 

In [126]:
input_username.send_keys("aaaa")


e o método `.clear()` para *limpar* o formulário:

In [ ]:
input_username.clear()


Como a variável `input_username` ainda é o mesmo elemento (o campo **Username**), podemos voltar a preencher o formulário:

In [128]:
nome_usuario = "pedrotechel"
input_username.send_keys(nome_usuario)


Elementos clicáveis, como botões ou checkboxes podem ser acessados através do método click:

In [130]:
input_opt1 = driver.find_element(
    By.XPATH, "/html/body/div/div[3]/form/table/tbody/tr[5]/td/input[1]"
)
input_opt1.click()


In [131]:
input_rdbx1 = driver.find_element(
    By.XPATH, "/html/body/div/div[3]/form/table/tbody/tr[6]/td/input[1]"
)
input_rdbx1.click()


## Exemplo Prático - Mercado Livre

Vamos construir um *webcrawler*, baseado em Selenium, para buscar preços de azeite no Mercado Livre. Plataformas de e-commerce quase sempre utilizam tecnologias que impossibilitam a utilização do BeautifulSoup!

Além de *apontar* o driver para a página principal do Mercado Livre, vamos utilizar o método `.implicitly_wait()` para que o driver *espere* um tempo até todos os elementos da página renderizarem:

In [132]:
driver.get("https://www.mercadolivre.com.br/")
driver.implicitly_wait(10)


Agora vamos interagir com a barra de busca para procurar azeites. Muitas barras de busca tem testos pré-preenchidos, então antes de continuarmos com nossa busca vamos limpar a barra utilizando o método `.clear()`:

In [133]:
barra_busca = driver.find_element(By.XPATH, "/html/body/header/div/form/input")
barra_busca.clear()


Com a barra limpa, podemos utilizar o método `.send_keys()` para preenche-la com o nome do produto que queremos buscar (`"azeite"`) e confirmar a busca utilizando o objeto `Keys.ENTER` (simulando digitar "azeite" e apertar *enter* na barra de busca): 

In [ ]:
barra_busca.send_keys("azeite")
barra_busca.send_keys(Keys.ENTER)


Até agora tranquilo! Para continuarmos, no entanto, precisaremos extrair TODOS os preços de produtos. Para isso utilizaremos o método `.find_elements()` (no plural) e, ao invés de utilizar o XPATH, buscaremos pelo **tag** e sua **classe**. Vamos começar encontrando todas as *caixas de produto*

In [139]:
lista_elem_produto = driver.find_elements(
    By.CLASS_NAME, "ui-search-result__content-wrapper"
)
len(lista_elem_produto)


56

A lista construída acima contém `WebElements` - da mesma forma que podemos fazer buscas dentro de **tags** hierarquicas utilizando o BS, podemos fazer buscas dentro de diferentes `WebElements` utilizando o Selenium:

In [140]:
lista_elem_preco = []
for produto in lista_elem_produto:
    caixa_preco = produto.find_element(By.CLASS_NAME, "price-tag-amount")
    lista_elem_preco.append(caixa_preco)


Vamos utilizar o atributo `.text` para extrair o conteúdo de cada uma das caixas de preço (primeiro testando com um elemento da lista):

In [147]:
preco_teste = lista_elem_preco[0]
preco_teste.text


'R$\n26\n,\n55'

O preço extraído acima parece não bater com o preço da página! No entanto, se prestarmos atenção, veremos que este é o preço sem desconto do produto (preço cheio).

Vamos alterar nosso loop para extrair todos os preços de cada produto (cheio e descontado):

In [159]:
lista_elem_preco = []
for produto in lista_elem_produto:
    caixa_preco = produto.find_elements(By.CLASS_NAME, "price-tag-amount")
    lista_elem_preco.append(caixa_preco)


In [160]:
lista_elem_preco[0]


[<selenium.webdriver.remote.webelement.WebElement (session="561981cdc99b53faa84c240fe6def5d7", element="9e441dbd-f34b-4c29-ae00-6cd77d6d1e01")>,
 <selenium.webdriver.remote.webelement.WebElement (session="561981cdc99b53faa84c240fe6def5d7", element="85abce4b-43d9-4931-be82-ecc2df65cfd9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="561981cdc99b53faa84c240fe6def5d7", element="4ba16bd7-1328-490a-9ed7-b8fa3e803547")>]

Temos 3 preços associados à este produto... Vamos analisar o que cada um deles representa:

In [164]:
[preco.text for preco in lista_elem_preco[0]]


['R$\n26\n,\n55', 'R$\n22\n,\n57', 'R$\n6\n,\n31']

O primeiro preço representa o preço cheio, o segundo com desconto e o terceiro é o preço de cada parcela. Este último não representa muita coisa, já que o número de parcelas pode variar entre produtos. Vamos tratar os strings dos dois primeiros, transformado-os em floats, em uma lista de preços:

In [154]:
import re


In [165]:
def limpar_preco(str_preco):
    pattern = r"[^0-9.,]"
    return float(re.sub(pattern, "", str_preco).replace(",", "."))


In [166]:
[limpar_preco(preco.text) for preco in lista_elem_preco[0]]


[26.55, 22.57, 6.31]

Agora vamos construir nosso loop!

In [178]:
lista_preco_cheio = []
lista_preco_desconto = []
pattern = r"[^0-9.,]"

for elem_preco in lista_elem_preco:
    precos = [limpar_preco(preco.text) for preco in elem_preco]
    lista_preco_cheio.append(precos[0])
    lista_preco_desconto.append(precos[1])

lista_preco_cheio


IndexError: list index out of range

In [186]:
lista_preco_cheio = []
lista_preco_desconto = []
pattern = r"[^0-9.,]"

for elem_preco in lista_elem_preco:
    if len(elem_preco) == 3:
        precos = [limpar_preco(preco.text) for preco in elem_preco]
        lista_preco_cheio.append(precos[0])
        lista_preco_desconto.append(precos[1])
    else:
        print("Preço fora do padrão!")
        lista_preco_cheio.append(np.nan)
        lista_preco_desconto.append(np.nan)


Preço fora do padrão!
Preço fora do padrão!
Preço fora do padrão!
Preço fora do padrão!
Preço fora do padrão!
Preço fora do padrão!
Preço fora do padrão!
Preço fora do padrão!
Preço fora do padrão!
Preço fora do padrão!
Preço fora do padrão!
Preço fora do padrão!
Preço fora do padrão!
Preço fora do padrão!


Podemos utilizar o mesmo método para construir a lista de nomes de produto:

In [187]:
lista_elem_produto[0].find_element(By.CLASS_NAME, "ui-search-item__title").text


'Azeite Chileno Extra Virgem O-live 500ml'

In [191]:
lista_nome = []
for elem_nome in lista_elem_produto:
    nome = elem_nome.find_element(By.CLASS_NAME, "ui-search-item__title")
    lista_nome.append(nome.text)


Agora vamos juntar nossas duas listas em um DataFrame:

In [193]:
tb_azeite = pd.DataFrame(
    {
        "nome": lista_nome,
        "preco_cheio": lista_preco_cheio,
        "preco_desconto": lista_preco_desconto,
    }
)
tb_azeite.head()


,nome,preco_cheio,preco_desconto
0,Azeite Chileno Extra Virgem O-live 500ml,26.55,22.57
1,Azeite Português Tipo Único Andorinha 500ml,32.05,27.24
2,Azeite de Oliva Extra Virgem Português Andorin...,31.19,22.49
3,Azeite de Oliva Extra Virgem Clássico Portuguê...,31.18,22.99
4,Azeite Espanhol Extra Virgem Borges 500ml,24.99,21.49
5,Azeite de Oliva Extra Virgem Português Herdade...,48.57,45.65
6,Azeite Chileno Extra Virgem O-live 500ml,26.55,22.57
7,Azeite de Oliva Extra Virgem Seleção Português...,31.49,26.77
8,Azeite de Oliva Extra Virgem Italiano De Cecco...,39.89,25.93
9,Azeite de Oliva Tipo Único Português Gallo Vid...,28.95,27.50
